# Requirement:
- numpy 1.19.5
- tensorflow 2.5.0
- tensorflow_privacy 0.6.1
- sklearn 0.24.2

Before performing the experiments, you need to fit the **mia_path** (path for MIA package) and **pickle_file**(path for tabular QMNIST data) according to your own PC.

In [1]:
import numpy as np

import pickle
import tensorflow as tf
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

# Load QMNIST data.

In [2]:
pickle_file = '/home/jiangnan/Desktop/dataset/QMNIST_tabular.pickle'

with open(pickle_file, 'rb') as f:
  pickle_data = pickle.load(f)
  x_defender = pickle_data['x_private']
  x_reserve = pickle_data['x_reserved']
  y_defender = pickle_data['y_private']
  y_reserve = pickle_data['y_reserved']
  del pickle_data
print('Data loaded.')

Data loaded.


In [3]:
NUM_CLASSES = 10

y_defender = y_defender[:,0]
y_reserve = y_reserve[:,0]

y_defender = np.expand_dims(y_defender,axis=1)
y_reserve = np.expand_dims(y_reserve,axis=1)

y_defender = tf.keras.utils.to_categorical(y_defender, num_classes=NUM_CLASSES)
y_reserve = tf.keras.utils.to_categorical(y_reserve, num_classes=NUM_CLASSES)

# Defender model $M_D$

In [4]:
#l2_norm_clip = 1.0
#noise_multiplier = 1.1

def defender_model_fn():
    """The architecture of the defender (victim) model.
    The attack is white-box, hence the attacker is assumed to know this architecture too."""
    
    tf.random.set_seed(10)
    model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    
    ##if train_with_DP:
    ##from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer
    ##train_op = DPKerasSGDOptimizer(
    ##    l2_norm_clip=l2_norm_clip,
    ##    noise_multiplier=noise_multiplier,
    ##    num_microbatches=1, # Possible problem after reducing the size of cost vector in tensorflow-privacy. Check: https://github.com/tensorflow/privacy/issues/17
    ##    learning_rate=1e-4
    ##    )
    ##else:
    
    train_op = tf.optimizers.Adam(1e-4)
    
    model.compile(optimizer=train_op,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


## Train $M_D$ 

In [5]:
number_records = 10000

# Train the defender model.
print("Training the defender model...")
defender_model = defender_model_fn()
defender_model.fit(
    x_defender[:number_records], y_defender[:number_records], epochs=20, batch_size = 32, validation_split=0.5, verbose=False
)

Training the defender model...


In [6]:
defender_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               65664     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 66,954
Trainable params: 66,954
Non-trainable params: 0
_________________________________________________________________


## Evaluate the utility of $M_D$ on utility evaluation dataset $E_U$ (which is equal to reserve dataset $R$).

Evaluation metrics: **Accuracy** & **AUC**

In [7]:
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import euclidean_distances

In [8]:
# compute the accuracy as acc
predict_reserve_proba = defender_model.predict_proba(x_reserve)
predict_reserve = np.argmax(predict_reserve_proba, axis=1)
label_reserve = np.argmax(y_reserve, axis=1)
acc = accuracy_score(label_reserve, predict_reserve)

/home/jiangnan/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


In [9]:
auc_by_class = []

# compute auc per class then take the average value
for i in range(NUM_CLASSES):
  class_indices = np.argmax(y_reserve, axis=1) == i
  fpr, tpr, thresholds = metrics.roc_curve(class_indices, predict_reserve_proba[:,i])
  auc = metrics.auc(fpr, tpr)
  auc_by_class.append(auc)

average_auc = np.mean(auc_by_class)

In [10]:
print('Utility of defender model:')
print('Acc: {}'.format(acc))
print('Auc: {}'.format(average_auc))

Utility of defender model:
Acc: 0.9308436724565757
Auc: 0.9959518907915067


In [11]:
# if necessary, save or reload the defender model trained

#defender_model_path = '/home/jiangnan/Desktop/model/QMNIST_defender_model'
#defender_model.save(defender_model_path)

#defender_model = tf.keras.models.load_model(defender_model_path)

# Oracle attack model $M_A$

In [11]:
import random
from tqdm import tqdm

In [12]:
data_in = x_defender[:number_records], y_defender[:number_records]
data_out = x_reserve[:number_records], y_reserve[:number_records]

In [13]:
random_index = random.randint(0,number_records-1)

evaluation_data_in = data_in[0][random_index]
evaluation_label_in = data_in[1][random_index]

evaluation_data_out = data_out[0][random_index]
evaluation_label_out = data_out[1][random_index]

evaluation_data = np.array([evaluation_data_in, evaluation_data_out])
evaluation_label = np.array([evaluation_label_in, evaluation_label_out])

evaluation = evaluation_data, evaluation_label


attack_train_data_in = np.delete(data_in[0], random_index, axis=0)
attack_train_label_in = np.delete(data_in[1], random_index, axis=0)

attack_in = attack_train_data_in, attack_train_label_in

attack_train_data_out = np.delete(data_out[0], random_index, axis=0)
attack_train_label_out = np.delete(data_out[1], random_index, axis=0)

attack_out = attack_train_data_out, attack_train_label_out

In [14]:
all_predict = []

for i in tqdm(range(100)):
    
    fake_with_out = np.vstack((attack_out[0][i],attack_in[0])), np.vstack((attack_out[1][i],attack_in[1]))

    fake_defender_model_out = defender_model_fn()
    fake_defender_model_out.fit(fake_with_out[0],fake_with_out[1], epochs=20, batch_size = 32, validation_split=0.5, verbose=False)
    
    out_predict = fake_defender_model_out.predict(attack_in[0])
    all_predict.append(out_predict)

100%|██████████| 100/100 [10:29<00:00,  6.29s/it]


In [15]:
predict = defender_model.predict(attack_in[0])
falseModelDists = []

for i in range(100):
    falseModelDists.append(np.mean(np.linalg.norm(all_predict[i]-predict, axis=1)))

threshold = np.percentile(np.array(falseModelDists),50)
print('threshold:{}'.format(threshold))

threshold:0.015848858281970024


In [16]:
predict = defender_model.predict(attack_in[0])

results = []
similarities = []

for i in tqdm(range(evaluation[0].shape[0])):
    
    attack_in_plus_one = np.vstack((evaluation[0][i].reshape(1,attack_in[0].shape[1]),attack_in[0])), np.vstack(( evaluation[1][i].reshape(1,attack_in[1].shape[1]),attack_in[1]))
    
    M_cD = defender_model_fn()
    M_cD.fit(attack_in_plus_one[0], attack_in_plus_one[1], epochs=20, batch_size = 32, validation_split=0.5, verbose=False)
    M_cD_predict = M_cD.predict(attack_in[0])
    similarity = np.mean(np.linalg.norm(M_cD_predict-predict, axis=1))
    similarities.append(similarity)
    
    if similarity < threshold:
        results.append(1)
        print('Point {} is in defender data'.format(i))
    else:
        results.append(0)
        print('Point {} is not in defender data'.format(i))

 50%|█████     | 1/2 [00:06<00:06,  6.18s/it]

Point 0 is not in defender data


100%|██████████| 2/2 [00:12<00:00,  6.18s/it]

Point 1 is in defender data


In [ ]:
similarities